In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.cluster.hierarchy as sch
import seaborn as sns
from scipy.stats import shapiro
from scipy.stats import norm
from scipy.optimize import minimize

df = pd.read_csv("./Clean/clean-huf-spy-6.csv")
df = df.drop(columns=['Unnamed: 0'])
df.set_index('Dátum', inplace=True)
df = df[::-1]
df = df[df.index >= '2018-03-09']
annual_risk_free_rate = 0.02
daily_risk_free_rate = (1 + annual_risk_free_rate)**(1/252) - 1

benchmark = 'S&P 500'
interesting = ['Citadella_Származtatott_Relative_Return', 'Erste_Stock_Hungary_Indexkövető_Részvény_A_Relative_Return',
               'SP500_Relative_Return', 'HOLD_Nemzetközi_Relative_Return', 'OTP_Supra_Relative_Return', 'HOLD_Columbus_Relative_Return']
# Create a dictionary to rename the columns to something simpler
rename_dict = {
    'Citadella_Származtatott': 'Citadella',
    'Erste_Stock_Hungary_Indexkövető_Részvény_A': 'Erste Indexkövető',
    'Generali_Arany_Oroszlán_ESG': 'Generali ESG',
    'HOLD_Columbus': 'HOLD Columbus',
    'HOLD_Nemzetközi': 'HOLD Nemzetközi',
    'HOLD_VM_Abszolút_Származtatott_Befektetési': 'HOLD Abszolút',
    'MBH_Fejlett_Piaci_Részvény': 'MBH Fejlett Piaci',
    'OTP_Klímaváltozás_13030': 'OTP Klímaváltozás',
    'OTP_Supra': 'OTP Supra',
    'VIG_Opportunity_Fejlett_Piaci_Részvény': 'VIG Opportunity',
    'Accorde_Abacus_Alap': 'Accord Abacus',
    'Accorde_Prizma_Alap': 'Accord Prizma',
    'HOLD_Orion': 'HOLD Orion',
    'OTP Föld Kincsei Árupiaci Alapok Alapja HUF': 'OTP Föld Kincsei',
    'Platina Delta Származtatott Befektetési Alap A':'Platina Delta',
    'Raiffeisen Részvény Alap':'Raiffeisen Részvény',
    'Takarék Származtatott Befektetési Alap': 'Gránit Takarék',
    'TakarékApolloSzármaztatottRészvény': 'Gránit Apollo',
    'SPY': 'S&P 500',
}

df_relative_returns = df.copy()
relative_return_columns = []
for column in df_relative_returns.columns:
    new_column = column + '_'
    df_relative_returns[column] = df_relative_returns[column] / df_relative_returns[column].iloc[0]
    
df=df_relative_returns
# df=df_relative_returns
df = df.rename(columns=rename_dict)

daily_returns = df.pct_change().dropna()


In [3]:
import scipy.stats as stats
from scipy.stats import kstest
from scipy.stats import levy_stable

results = {}
for column in daily_returns.columns:
    pconv = lambda alpha, beta, mu, sigma: (alpha, beta, mu - sigma * beta * np.tan(np.pi * alpha / 2.0), sigma)
    params = pconv(*levy_stable._fitstart(daily_returns[column]))
    ks_stat, p_value = kstest(daily_returns[column], 'levy_stable', args=params)
    bootstrap_ks_stats = []
    for iter in range(100):
        resample = stats.levy_stable.rvs(*params, size=len(daily_returns[column]))
        bootstrap_ks_stat, _ = kstest(resample, 'levy_stable', args=params)
        bootstrap_ks_stats.append(bootstrap_ks_stat)
    ks_95th_percentile = np.percentile(bootstrap_ks_stats, 95)
    is_above_95th = ks_stat > ks_95th_percentile
    results[column] = {
        'Params': params,
        'KS Stat': ks_stat,
        'p-value': p_value,
        '95th Percentile': ks_95th_percentile,
        'Is Above 95th': is_above_95th
    }
    print(results[column])

results_df = pd.DataFrame(results).T

{'Params': (1.4484220498254003, -0.022271496464045602, 0.0002785329995735261, 0.0023189592638141717), 'KS Stat': 0.018760062230179453, 'p-value': 0.6421487819385092, '95th Percentile': 0.0359578571819447, 'Is Above 95th': False}
{'Params': (1.5946701062286048, -0.03831011788993581, 0.0002517122299197768, 0.00098585900091827), 'KS Stat': 0.02220579630390429, 'p-value': 0.4260109908904627, '95th Percentile': 0.03367401154935337, 'Is Above 95th': False}
{'Params': (1.60205726135326, 0.06245112060776427, 0.0012916107574200183, 0.0069695793743419976), 'KS Stat': 0.024997997084967083, 'p-value': 0.28511304573446616, '95th Percentile': 0.03288304014153605, 'Is Above 95th': False}
{'Params': (1.599067487059083, 0.09408636875243852, 0.0004856116521954678, 0.002556055954152191), 'KS Stat': 0.0231114330905976, 'p-value': 0.3762970624355437, '95th Percentile': 0.03352181974139022, 'Is Above 95th': False}
{'Params': (1.5499528088992383, -0.009535829366717116, 0.0008095599232416206, 0.00678413983439

In [4]:
results_df.to_csv('./Clean/stable_fit.csv')